In [13]:
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))



df = pd.read_csv(os.path.join(os.getcwd(), "../../data/total_4am.csv"), index_col=0)


In [14]:
import pandas as pd
from performance import get_performance

def various_ma(interval_list, df):

    original_df = df.copy()  # Make a copy of the original DataFrame

    for interval in interval_list:
        for interval2 in interval_list:
            if interval < interval2:
                # break

                df = original_df.copy()  # Create a fresh copy for each interval

                df[f"{interval}_ma"] = df['close'].rolling(window=interval).mean()
                df[f'{interval2}_ma'] = df['close'].rolling(window=interval2).mean()

                # Implement RSI strategy for long positions only
                df['signal'] = 0  # Default to no position
                for i in range(200, len(df)):
                    # Buy condition
                    # if df['close'].iloc[i] >= df[f'{interval}_ma'].iloc[i]:
                    if df[f'{interval}_ma'].iloc[i] >= df[f'{interval2}_ma'].iloc[i]:
                        df.loc[i, 'signal'] = 1
                    # Sell condition
                    elif df[f'{interval}_ma'].iloc[i] < df[f'{interval2}_ma'].iloc[i]:
                        df.loc[i, 'signal'] = -1

                # Manage positions
                df['position'] = 0
                holding_position = False

                for i in range(1, len(df)):
                    if df['signal'].iloc[i] == 1 and not holding_position:
                        df.loc[i, 'position'] = 1
                        holding_position = True
                    elif df['signal'].iloc[i] == -1 and holding_position:
                        df.loc[i, 'position'] = 0
                        holding_position = False
                    else:
                        df.loc[i, 'position'] = df['position'].iloc[i-1]

                # Calculate the strategy returns (only when in a long position)
                df['strategy_returns'] = df['position'].shift(1) * df['close'].pct_change()

                # Calculate the cumulative returns
                df['cumulative_returns'] = (1 + df['strategy_returns']).cumprod()

                # Calculate the benchmark cumulative returns (buy and hold strategy)
                df['benchmark_returns'] = (1 + df['close'].pct_change()).cumprod()

                # Save the results to a CSV file
                df.to_csv(f"../../results/ma/ma_crossover({interval})_({interval2}).csv")

                # Get performance metrics and plot the results
                get_performance(df=df, title=f"ma_crossover({interval})_({interval2})")

                
            df = original_df.copy()  # Create a fresh copy for each interval


In [15]:
interval_list = [5, 10, 20, 50, 90, 120, 200]
various_ma(interval_list=interval_list, df=df)

======Investment Summary======
Strategy        : ma_crossover(5)_(10)
total_return    : 56.35
cagr            : 17.04
mdd             : 51.57
investing_days  : 1037
======Investment Summary======
Strategy        : ma_crossover(5)_(20)
total_return    : 40.94
cagr            : 12.84
mdd             : 54.17
investing_days  : 1037
======Investment Summary======
Strategy        : ma_crossover(5)_(50)
total_return    : 50.32
cagr            : 15.43
mdd             : 45.21
investing_days  : 1037
======Investment Summary======
Strategy        : ma_crossover(5)_(90)
total_return    : 21.54
cagr            : 7.11
mdd             : 51.08
investing_days  : 1037
======Investment Summary======
Strategy        : ma_crossover(5)_(120)
total_return    : 65.96
cagr            : 19.52
mdd             : 42.36
investing_days  : 1037
======Investment Summary======
Strategy        : ma_crossover(5)_(200)
total_return    : 58.69
cagr            : 17.65
mdd             : 35.28
investing_days  : 1037
======Inv